# 0. Import required libraries & setting parameters

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import random as rnd
import plotly.express as px 

from sklearn import datasets
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC

import datetime

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

sns.set(rc={"figure.dpi":100, 'savefig.dpi':300})
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats
%config InlineBackend.figure_format = 'retina'
plt.style.use('tableau-colorblind10')

# 1. Data cleaning

## 1. Reading the dataset

In [2]:
df = pd.read_csv('Gurugram_Hourly_Compiled.csv')
df

/var/folders/27/b03t8q596qvcmtyqjtnh0w6w0000gn/T/ipykernel_88558/2512706165.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,25,26,27,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Gurugram_Hourly_Compiled.csv')


,Datetime,Station,PM2.5,NO,NO2,NOx,SO2,CO,Ozone,Benzene,...,Xylene,AT,RF,TOT-RF,PM10,NH3,CO2,O3,AQI,AQI_Bucket
0,04-03-2020 00:00,Vikas Sadan,60.92,141.22,61.53,144.62,10.5,3.01,5.08,2.04,...,3.1,None,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,04-03-2020 01:00,Vikas Sadan,56.88,177.38,56.39,170.94,5.64,5.38,6.15,2,...,3.03,None,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,04-03-2020 02:00,Vikas Sadan,63.33,302.33,40.26,262.69,7.6,1.85,12.72,2.02,...,3.08,None,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,04-03-2020 03:00,Vikas Sadan,67.6,377.93,29.18,317.53,10.53,1.31,11.46,1.96,...,3.02,None,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,04-03-2020 04:00,Vikas Sadan,72.72,319.8,33.66,273.25,9.98,0.63,4.76,2.01,...,2.97,None,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147526,2020-06-30 20:00:00,NaN,54.74,6.23,10.67,12.6,4.18,0.6,NaN,2.14,...,4.7,NaN,NaN,NaN,135.42,2.95,NaN,70.95,154.0,Moderate
147527,2020-06-30 21:00:00,NaN,54.37,6.3,10.28,11.88,4.03,1.2,NaN,1.87,...,4.29,NaN,NaN,NaN,136.11,3.47,NaN,55.13,154.0,Moderate
147528,2020-06-30 22:00:00,NaN,50.34,6.93,10.36,13.01,6.17,1.0,NaN,2.41,...,4.59,NaN,NaN,NaN,147.32,3.76,NaN,42.60,155.0,Moderate
147529,2020-06-30 23:00:00,NaN,53.23,6.64,10.3,12.75,6.31,0.86,NaN,2.54,...,4.61,NaN,NaN,NaN,163.15,3.36,NaN,35.61,156.0,Moderate


In [3]:
df.shape

(147531, 31)

## 2. Looking into the dataset

In [4]:
df.head(5)

,Datetime,Station,PM2.5,NO,NO2,NOx,SO2,CO,Ozone,Benzene,...,Xylene,AT,RF,TOT-RF,PM10,NH3,CO2,O3,AQI,AQI_Bucket
0,04-03-2020 00:00,Vikas Sadan,60.92,141.22,61.53,144.62,10.5,3.01,5.08,2.04,...,3.1,None,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,04-03-2020 01:00,Vikas Sadan,56.88,177.38,56.39,170.94,5.64,5.38,6.15,2,...,3.03,None,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,04-03-2020 02:00,Vikas Sadan,63.33,302.33,40.26,262.69,7.6,1.85,12.72,2.02,...,3.08,None,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,04-03-2020 03:00,Vikas Sadan,67.6,377.93,29.18,317.53,10.53,1.31,11.46,1.96,...,3.02,None,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,04-03-2020 04:00,Vikas Sadan,72.72,319.8,33.66,273.25,9.98,0.63,4.76,2.01,...,2.97,None,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.tail(5)

,Datetime,Station,PM2.5,NO,NO2,NOx,SO2,CO,Ozone,Benzene,...,Xylene,AT,RF,TOT-RF,PM10,NH3,CO2,O3,AQI,AQI_Bucket
147526,2020-06-30 20:00:00,NaN,54.74,6.23,10.67,12.6,4.18,0.6,NaN,2.14,...,4.7,NaN,NaN,NaN,135.42,2.95,NaN,70.95,154.0,Moderate
147527,2020-06-30 21:00:00,NaN,54.37,6.3,10.28,11.88,4.03,1.2,NaN,1.87,...,4.29,NaN,NaN,NaN,136.11,3.47,NaN,55.13,154.0,Moderate
147528,2020-06-30 22:00:00,NaN,50.34,6.93,10.36,13.01,6.17,1.0,NaN,2.41,...,4.59,NaN,NaN,NaN,147.32,3.76,NaN,42.60,155.0,Moderate
147529,2020-06-30 23:00:00,NaN,53.23,6.64,10.3,12.75,6.31,0.86,NaN,2.54,...,4.61,NaN,NaN,NaN,163.15,3.36,NaN,35.61,156.0,Moderate
147530,2020-07-01 00:00:00,NaN,61.64,5.99,10.58,12.37,7.57,1.11,NaN,2.67,...,4.73,NaN,NaN,NaN,174.08,2.91,NaN,44.14,157.0,Moderate


In [6]:
df.sample(5)

,Datetime,Station,PM2.5,NO,NO2,NOx,SO2,CO,Ozone,Benzene,...,Xylene,AT,RF,TOT-RF,PM10,NH3,CO2,O3,AQI,AQI_Bucket
36276,02-04-2021 04:00,Sector 51,94.73,16.04,2.97,8.08,3.43,0.54,25,6.4,...,3.57,21.44,0,0.0,227.72,30.22,NaN,NaN,NaN,NaN
63767,29-04-2021 22:00,NISE,49.9,4.34,6.82,11.16,NaN,0.58,45.85,NaN,...,NaN,NaN,None,0.0,207.42,NaN,None,NaN,NaN,NaN
15580,13-12-2021 04:00,Vikas Sadan,179.3,230.8,58.08,205.82,8.31,2.83,66.57,4.01,...,1.93,None,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
7663,17-01-2021 07:00,Vikas Sadan,119.72,6.74,40.7,26.7,0.91,1.47,7.51,1.98,...,3.76,None,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN
36353,05-04-2021 09:00,Sector 51,460.27,7.96,13.09,17.51,3.91,0.91,39.2,10.03,...,5.53,31.48,0,0.0,506.63,40.52,NaN,NaN,NaN,NaN


### Checking the datatypes

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147531 entries, 0 to 147530
Data columns (total 31 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Datetime     147531 non-null  object 
 1   Station      107273 non-null  object 
 2   PM2.5        141387 non-null  object 
 3   NO           140785 non-null  object 
 4   NO2          141476 non-null  object 
 5   NOx          134829 non-null  object 
 6   SO2          112070 non-null  object 
 7   CO           142368 non-null  object 
 8   Ozone        107273 non-null  object 
 9   Benzene      108032 non-null  object 
 10  Toluene      104172 non-null  object 
 11  Eth-Benzene  80457 non-null   object 
 12  MP-Xylene    80457 non-null   object 
 13  O            26816 non-null   object 
 14  Temp         26816 non-null   object 
 15  RH           80457 non-null   object 
 16  WS           53641 non-null   object 
 17  WD           80457 non-null   object 
 18  SR           80457 non-n

In [8]:
df.columns.duplicated().sum()

0

In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
TOT-RF,107273.0,0.070499,2.380372,0.00,0.00,0.00,0.00,246.81
O3,34793.0,34.747413,31.186317,0.01,11.33,23.96,50.36,199.80
AQI,33420.0,223.877409,121.771297,31.00,119.00,205.00,321.00,966.00


In [10]:
df.describe(include='object').T

,count,unique,top,freq
Datetime,147531,67083,04-03-2020 00:00,4
Station,107273,4,Sector 51,26825
PM2.5,141387,36863,None,11113
NO,140785,13883,None,10697
NO2,141476,14463,None,9863
NOx,134829,16927,None,7613
SO2,112070,7891,None,8535
CO,142368,1677,None,7541
Ozone,107273,13252,None,9341
Benzene,108032,3697,None,16015


In [11]:
df.describe(include='object').T.shape
df.describe(include='float').T.shape

(28, 4)

(3, 8)

In [12]:
for i in df.columns:
    print(i,"has",df[i].isnull().sum(),"missing values")
    print()

Datetime has 0 missing values

Station has 40258 missing values

PM2.5 has 6144 missing values

NO has 6746 missing values

NO2 has 6055 missing values

NOx has 12702 missing values

SO2 has 35461 missing values

CO has 5163 missing values

Ozone has 40258 missing values

Benzene has 39499 missing values

Toluene has 43359 missing values

Eth-Benzene has 67074 missing values

MP-Xylene has 67074 missing values

O has 120715 missing values

Temp has 120715 missing values

RH has 67074 missing values

WS has 93890 missing values

WD has 67074 missing values

SR has 67074 missing values

BP has 120715 missing values

VWS has 120715 missing values

Xylene has 64257 missing values

AT has 67074 missing values

RF has 40258 missing values

TOT-RF has 40258 missing values

PM10 has 50524 missing values

NH3 has 91168 missing values

CO2 has 120715 missing values

O3 has 112738 missing values

AQI has 114111 missing values

AQI_Bucket has 114111 missing values



In [26]:
df_describe_object = df.describe(include=['object','float']).T
l = []
for i in df_describe_object.index:
    l.append(df[i].isnull().sum())
df_describe_object["Missing values"] = l

In [27]:
df_describe_object = df_describe_object[['count', 'Missing values', 'unique', 'top', 'freq']]
df_describe_object

,count,Missing values,unique,top,freq
Datetime,147531,0,67083,04-03-2020 00:00,4
Station,107273,40258,4,Sector 51,26825
PM2.5,141387,6144,36863,None,11113
NO,140785,6746,13883,None,10697
NO2,141476,6055,14463,None,9863
NOx,134829,12702,16927,None,7613
SO2,112070,35461,7891,None,8535
CO,142368,5163,1677,None,7541
Ozone,107273,40258,13252,None,9341
Benzene,108032,39499,3697,None,16015


In [25]:
# df.info()
len(df.columns)
len(df_describe_object.index)
df.describe(include=['object','float']).T

31

28

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Datetime,147531,67083,04-03-2020 00:00,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Station,107273,4,Sector 51,26825,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PM2.5,141387,36863,None,11113,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NO,140785,13883,None,10697,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NO2,141476,14463,None,9863,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NOx,134829,16927,None,7613,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SO2,112070,7891,None,8535,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CO,142368,1677,None,7541,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ozone,107273,13252,None,9341,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Benzene,108032,3697,None,16015,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_describe_object['Missing %'] = round((df_describe_object['Missing values'] /
                                   df.shape[0]) * 100, 2)
df_describe_object

,count,Missing values,unique,top,freq,Missing %
Datetime,147531,0,67083,04-03-2020 00:00,4,0.00
Station,107273,40258,4,Sector 51,26825,27.29
PM2.5,141387,6144,36863,None,11113,4.16
NO,140785,6746,13883,None,10697,4.57
NO2,141476,6055,14463,None,9863,4.10
NOx,134829,12702,16927,None,7613,8.61
SO2,112070,35461,7891,None,8535,24.04
CO,142368,5163,1677,None,7541,3.50
Ozone,107273,40258,13252,None,9341,27.29
Benzene,108032,39499,3697,None,16015,26.77


In [29]:
df_describe_object.sort_values(by = ['Missing %'], ascending = False).style.bar()

,count,Missing values,unique,top,freq,Missing %
O,26816,120715,3496,None,4600,81.820000
CO2,26816,120715,1,None,26816,81.820000
VWS,26816,120715,71,None,4596,81.820000
BP,26816,120715,5520,None,5000,81.820000
Temp,26816,120715,7398,None,4594,81.820000
AQI_Bucket,33420,114111,6,Moderate,10271,77.350000
AQI,33420.000000,114111,nan,nan,nan,77.350000
O3,34793.000000,112738,nan,nan,nan,76.420000
WS,53641,93890,316,None,942,63.640000
NH3,56363,91168,13959,None,5677,61.800000


In [ ]:
px.bar(df_describe_object['Missing %'].sort_values(ascending=False),
       text_auto=True)

In [31]:
df

,Datetime,Station,PM2.5,NO,NO2,NOx,SO2,CO,Ozone,Benzene,...,Xylene,AT,RF,TOT-RF,PM10,NH3,CO2,O3,AQI,AQI_Bucket
0,04-03-2020 00:00,Vikas Sadan,60.92,141.22,61.53,144.62,10.5,3.01,5.08,2.04,...,3.1,None,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,04-03-2020 01:00,Vikas Sadan,56.88,177.38,56.39,170.94,5.64,5.38,6.15,2,...,3.03,None,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,04-03-2020 02:00,Vikas Sadan,63.33,302.33,40.26,262.69,7.6,1.85,12.72,2.02,...,3.08,None,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,04-03-2020 03:00,Vikas Sadan,67.6,377.93,29.18,317.53,10.53,1.31,11.46,1.96,...,3.02,None,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,04-03-2020 04:00,Vikas Sadan,72.72,319.8,33.66,273.25,9.98,0.63,4.76,2.01,...,2.97,None,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147526,2020-06-30 20:00:00,NaN,54.74,6.23,10.67,12.6,4.18,0.6,NaN,2.14,...,4.7,NaN,NaN,NaN,135.42,2.95,NaN,70.95,154.0,Moderate
147527,2020-06-30 21:00:00,NaN,54.37,6.3,10.28,11.88,4.03,1.2,NaN,1.87,...,4.29,NaN,NaN,NaN,136.11,3.47,NaN,55.13,154.0,Moderate
147528,2020-06-30 22:00:00,NaN,50.34,6.93,10.36,13.01,6.17,1.0,NaN,2.41,...,4.59,NaN,NaN,NaN,147.32,3.76,NaN,42.60,155.0,Moderate
147529,2020-06-30 23:00:00,NaN,53.23,6.64,10.3,12.75,6.31,0.86,NaN,2.54,...,4.61,NaN,NaN,NaN,163.15,3.36,NaN,35.61,156.0,Moderate
